In [52]:
import pandas as pd
import numpy as np
import datetime
import requests
from hanspell import spell_checker
from kss import split_sentences

In [62]:
# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran

### 전처리 함수 설정

맞춤법 교정 함수(500자 이상 포함)

In [ ]:
def spell_check_all(txt):
    from hanspell import spell_checker
    checked_list = []
    if len(txt) >= 500:
        sentences = split_sentences(txt)
        for sen in sentences:
            checked_list.append(spell_checker.check(sen).checked)
        result = ' '.join(checked_list)
    else:
        result = spell_checker.check(txt).checked
    return result

토큰화 함수(konlpy)

In [61]:
# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출한다.
def get_pos (x) :
    tagger = Okt()
    pos = tagger.pos(x)
    #pos = tagger.pos(x, norm = True, stem = True)
    return pos

In [2]:
kurly_data = pd.read_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kurly_data.csv", index_col=0)
workerbee_data = pd.read_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\workerbee_data.csv", index_col=0)
kakao_data = pd.read_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kakao_data.csv", index_col=0)

### 마켓컬리 데이터 전처리

In [67]:
kurly_data.head(2)

,ProductCode,MemberGrade,MemberName,ProductName,ReviewText,ReviewDate,ReviewLike
0,5136210,일반,박**,[바로생활] 아카시아꿀 스틱 10포,작은 박스로 2개 (10개씩 들어있어 20개네요 ) 주문했습니다. 의심이 많은 타...,2023.06.23,0
1,5136210,프렌즈,강**,[바로생활] 아카시아꿀 스틱 10포,물에 타먹으니까 되게 다네요,2023.08.15,0


줄바꿈 기호 변환 / '&'기호 ','로 대체

In [4]:
kurly_data['ReviewText'] = kurly_data['ReviewText'].apply(lambda x:x.replace('\n', ' ') if '\n' in x else x)

In [ ]:
kurly_data['ReviewText'] = kurly_data['ReviewText'].apply(lambda x:x.replace('&', ', ') if '&' in x else x)

맞춤법 교정

In [68]:
kurly_data['ReviewText'] = kurly_data['ReviewText'].apply(lambda x:spell_check_all(x))

토큰화

In [69]:
kurly_data['token_okt'] = kurly_data['ReviewText'].apply(lambda x:get_pos(x))

In [70]:
kurly_data.head(2)

,ProductCode,MemberGrade,MemberName,ProductName,ReviewText,ReviewDate,ReviewLike,token_okt
0,5136210,일반,박**,[바로생활] 아카시아꿀 스틱 10포,작은 박스로 2개 (10개씩 들어있어 20개네요 ) 주문했습니다. 의심이 많은 타...,2023.06.23,0,"[(작은, Adjective), (박스, Noun), (로, Josa), (2, N..."
1,5136210,프렌즈,강**,[바로생활] 아카시아꿀 스틱 10포,물에 타먹으니까 되게 다네요,2023.08.15,0,"[(물, Noun), (에, Josa), (타, Noun), (먹으니까, Verb)..."


In [97]:
kurly_data.to_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kurly_data_preprocessed.csv")

### 워커비 데이터 전처리

In [71]:
workerbee_data.head(2)

,itemNo,itemName,lastSalePrice,heartCount,reviewCount,itemReviewNo,optionValue,userId,point,contents,reviewType,registrationType,isGift,isReported,giftReview,partnerComment,insertTimestamp
0,1308881,[꿀카라멜] 워커비 허니 얼그레이 카라멜 50g,3000,7126,796,6390415,['[구매단위]박스구매(12개입)'],hai***,5,"°고마운 일이 생겨, 지인분께 선물로도 드리고~ 나도 먹고! 재주문 했어요~, 포장...",0,USER,F,F,NaN,NaN,2023-08-11 05:42:48
1,1308881,[꿀카라멜] 워커비 허니 얼그레이 카라멜 50g,3000,7126,796,6376671,['[구매단위]낱개구매'],yangsfami***,5,얼그레이향이 은은하게 나서 정말 맛있어요 당 떨어질 때 하나씩 집어먹기 딱 좋은 것...,0,USER,F,F,NaN,NaN,2023-08-09 16:58:54


In [82]:
print(workerbee_data['reviewType'].value_counts())
print(workerbee_data['registrationType'].value_counts())
print(workerbee_data['isGift'].value_counts())
print(workerbee_data['isReported'].value_counts())

0    5110
Name: reviewType, dtype: int64
USER    5110
Name: registrationType, dtype: int64
F    3925
T    1185
Name: isGift, dtype: int64
F    5110
Name: isReported, dtype: int64


In [81]:
print(len(workerbee_data['giftReview'].isna()))
print(len(workerbee_data['partnerComment'].isna()))

5110
5110


결측치 컬럼 제거

In [84]:
workerbee_data.drop(['reviewType', 'registrationType', 'isReported', 'giftReview', 'partnerComment'], axis = 1, inplace=True)

줄바꿈 기호 변환 / '&' 기호 ','로 대체

In [88]:
workerbee_data['contents'] = workerbee_data['contents'].apply(lambda x:x.replace('\n', ' ') if '\n' in x else x)

In [90]:
workerbee_data['contents'] = workerbee_data['contents'].apply(lambda x:x.replace('&', ', ') if '&' in x else x)

맞춤법 교정

In [91]:
workerbee_data['contents'] = workerbee_data['contents'].apply(lambda x:spell_check_all(x))

토큰화

In [94]:
workerbee_data['token_okt'] = workerbee_data['contents'].apply(lambda x:get_pos(x))

In [95]:
workerbee_data.head(2)

,itemNo,itemName,lastSalePrice,heartCount,reviewCount,itemReviewNo,optionValue,userId,point,contents,isGift,insertTimestamp,token_okt
0,1308881,[꿀카라멜] 워커비 허니 얼그레이 카라멜 50g,3000,7126,796,6390415,['[구매단위]박스구매(12개입)'],hai***,5,"°고마운 일이 생겨, 지인분께 선물로도 드리고~ 나도 먹고! 재주문 했어요~, 포장...",F,2023-08-11 05:42:48,"[(°, Foreign), (고마운, Adjective), (일이, Modifier..."
1,1308881,[꿀카라멜] 워커비 허니 얼그레이 카라멜 50g,3000,7126,796,6376671,['[구매단위]낱개구매'],yangsfami***,5,얼 그레이 향임 은은하게 나서 정말 맛있어요 당 떨어질 때 하나씩 집어먹기 딱 좋은...,F,2023-08-09 16:58:54,"[(얼, Noun), (그레이, Noun), (향임, Noun), (은은하게, Ad..."


In [96]:
workerbee_data.to_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\workerbee_data_preprocessed.csv")

### 카카오톡 선물하기 데이터 전처리

In [44]:
kakao_data = pd.read_csv(r"C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kakao_data.csv", index_col=0)

In [45]:
kakao_data.head(2)

,id,name,brand,sellingPrice,wishCount,reviewCount,product_tag1,product_tag2,review_text,rating,review_tag1,review_tag2,review_time
0,2270784,국산 프로폴리스 혼합 천연벌꿀 허니스틱(15스틱) 답례품 비마스터 -23.5,비마스터,27800,2228,1152,[],[],숙취후 아침에 따뜻한 물에 한포 간편히 타먹을수 있어서 좋드라구요.\n스틱형이라 휴...,4,응원,직장동료,20230410141648
1,2270784,국산 프로폴리스 혼합 천연벌꿀 허니스틱(15스틱) 답례품 비마스터 -23.5,비마스터,27800,2228,1152,[],[],친구가 건강관리 하라고 선물해 주었습니다 !!\n후기가 좋아서 기대했는데\n맛있고 ...,4,생일,친구,20230409032702


줄바꿈 기호 변환

In [46]:
kakao_data['review_text'] = kakao_data['review_text'].apply(lambda x:x.replace('\n', ' ') if '\n' in x else x)

리뷰 작성 시각 날짜형식으로 변환

In [47]:
kakao_data['review_time'][0]

20230410141648

In [48]:
kakao_data['review_time'] = kakao_data['review_time'].apply(lambda x:datetime.datetime.strptime(str(x), '%Y%m%d%H%M%S'))

In [49]:
kakao_data['review_time'][0]

Timestamp('2023-04-10 14:16:48')

결측치 컬럼 확인 후 제거

In [42]:
kakao_data.describe()

,id,sellingPrice,wishCount,reviewCount,rating
count,1.922000e+03,1922.000000,1922.000000,1922.000000,1922.000000
mean,2.384494e+06,27667.637877,2204.285640,661.895942,3.929240
std,1.210880e+06,10013.266241,1733.916523,516.820952,0.363894
min,7.586040e+05,7900.000000,2.000000,1.000000,1.000000
25%,2.144896e+06,25900.000000,973.000000,77.000000,4.000000
50%,2.270784e+06,27800.000000,2228.000000,1152.000000,4.000000
75%,2.270784e+06,27800.000000,2228.000000,1152.000000,4.000000
max,7.280901e+06,85900.000000,5840.000000,1152.000000,4.000000


In [50]:
kakao_data['product_tag1'].value_counts()

[]    1922
Name: product_tag1, dtype: int64

In [36]:
kakao_data.drop('product_tag1', axis = 1, inplace=True)

tag2는 264개 데이터가 package_gift로 존재

In [51]:
kakao_data['product_tag2'].value_counts()

[]                  1658
['PACKAGE_GIFT']     264
Name: product_tag2, dtype: int64

맞춤법 교정

In [53]:
kakao_data['review_text'] = kakao_data['review_text'].apply(lambda x:x.replace('&', ', ') if '&' in x else x)

In [55]:
kakao_data['review_text'] = kakao_data['review_text'].apply(lambda x:spell_check_all(x))

토큰화

In [64]:
kakao_data['token_okt'] = kakao_data['review_text'].apply(lambda x:get_pos(x))

In [65]:
kakao_data.head(1)

,id,name,brand,sellingPrice,wishCount,reviewCount,product_tag1,product_tag2,review_text,rating,review_tag1,review_tag2,review_time,token_okt
0,2270784,국산 프로폴리스 혼합 천연벌꿀 허니스틱(15스틱) 답례품 비마스터 -23.5,비마스터,27800,2228,1152,[],[],숙취 후 아침에 따뜻한 물에 한포 간편히 타먹을 수 있어서 좋더라고요. 스틱형이라 ...,4,응원,직장동료,2023-04-10 14:16:48,"[(숙취, Noun), (후, Noun), (아침, Noun), (에, Josa),..."


In [66]:
kakao_data.to_csv(r'C:\Users\NT550-048\Desktop\잇더\꿀스틱\data\kakao_data_preprocessed.csv')